In [31]:
import pandas as pd
from itertools import chain
import numpy as np

In [32]:
meses = [
'ENERO',
'FEBRERO',
'MARZO',
'ABRIL',
 'MAYO',
 'JUNIO',
 'JULIO',
 'AGOSTO',
 'SEPTIEMBRE',
 'OCTUBRE',
 'NOVIEMBRE',
 'DICIEMBRE']
rangosHora = None
year = '2015'

Importación de datos de excel desde 2013 - 2019

In [33]:
def tratarHoja (hoja):   
    global rangosHora 
    hoja = hoja.iloc[1:, 1:len(hoja.columns)-1]
    hoja = hoja.T
    hoja.columns = hoja.iloc[0]
    colTotal = list(hoja.columns).index('TOTAL')
    hoja = hoja.iloc[1:,0:colTotal]
    rangosHora = [s.replace(" ", "").replace("-", "a") for s in hoja.columns[1:7]]
    hoja.columns = ['Dia'] + rangosHora
    return hoja

In [34]:
def resumirHoja(hoja,mes,year):
    global rangosHora
    hoja['Personas'] = [tuple(x) for x in (hoja[rangosHora]).values]
    hoja['Dia'] = hoja['Dia'].astype(int).astype(str)
    lens = list(map(len, hoja.Personas))
    res = pd.DataFrame({'Dia': np.repeat(hoja.Dia,lens),
                        'Mes':np.tile([mes],len(lens)*len(rangosHora)),
                        'Año':np.tile([year],len(lens)*len(rangosHora)),
                        'RangoHora': np.tile(rangosHora,len(lens)),
                        'Aforo': list(chain.from_iterable(hoja.Personas))})
    return res

In [41]:
def iterarAnio(year):    
    xlsx = pd.ExcelFile('../Aforo Biblioteca Proyecto Maestría/Ingreso a edificio '+year+'.xlsx', engine='openpyxl')
    hojas = [x.upper() for x in xlsx.sheet_names if x in meses]
    pdMesesResumidos = []
    for mes in hojas:
        hoja = pd.read_excel(xlsx, mes)
        hoja = tratarHoja(hoja)
        pdMesesResumidos.append(resumirHoja(hoja,mes, year))
    return pd.concat(pdMesesResumidos)

In [43]:
anios = list(range(2013,2020))
pdList = []
for anio in anios:
    pdList.append(iterarAnio(str(anio)))
resumenCompleto = pd.concat(pdList, ignore_index=True)
print(resumenCompleto)

      Dia        Mes   Año RangoHora  Aforo
0       1      ABRIL  2013       0a6   34.0
1       1      ABRIL  2013       6a9  360.0
2       1      ABRIL  2013      9a14  267.0
3       1      ABRIL  2013     14a18  106.0
4       1      ABRIL  2013     18a24    0.0
...    ..        ...   ...       ...    ...
11818  14  DICIEMBRE  2019     6a9am    NaN
11819  14  DICIEMBRE  2019     9a2pm    NaN
11820  14  DICIEMBRE  2019     2a6pm    NaN
11821  14  DICIEMBRE  2019     6a8pm    NaN
11822  14  DICIEMBRE  2019    8a12pm    NaN

[11823 rows x 5 columns]
